In [16]:
%load_ext autoreload
%autoreload
import sys
sys.path.append("../")

from qiskit.algorithms import NumPyMinimumEigensolver
from qiskit_optimization.algorithms import PBILGroverOptimizer, GroverOptimizer, MinimumEigenOptimizer
from qiskit_optimization.problems import QuadraticProgram
from qiskit_optimization.translators import from_docplex_mp
from qiskit import Aer
from docplex.mp.model import Model
import numpy as np
backend = Aer.get_backend('aer_simulator')
backend.set_options(device='GPU')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
model = Model()
xs = [model.binary_var(name=f"x{i}") for i in range(9)]
objective = sum([np.random.randint(0, 3) * x for x in xs]) + sum([np.random.randint(-2, 3) * x * y for x in xs for y in xs])
# objective = (-xs[0]+2*xs[1]-3*xs[2]-2*xs[0]*xs[2]-1*xs[1]*xs[2])
model.minimize(objective)
qp = from_docplex_mp(model)
print(qp.prettyprint())

Problem name: docplex_model6

Minimize
  2*x0^2 + 3*x0*x1 + 2*x0*x2 - 3*x0*x3 - 3*x0*x4 + 2*x0*x5 - 2*x0*x7 - 2*x0*x8
  + x1^2 + x1*x2 + 2*x1*x3 + x1*x4 - x1*x5 + x1*x6 - 3*x1*x7 - 4*x1*x8 - 2*x2^2
  - x2*x3 - 4*x2*x4 - 2*x2*x5 + 2*x2*x6 - 3*x2*x8 + 4*x3*x4 - x3*x7 + x3*x8
  - x4^2 + 2*x4*x5 + 4*x4*x6 - 2*x4*x7 + 3*x4*x8 + 2*x5*x6 - 4*x5*x8 + 2*x6^2
  + x6*x7 - 3*x6*x8 - x7^2 + 2*x8^2 + 2*x0 + 2*x4 + x5 + 2*x6 + x7 + 2*x8

Subject to
  No constraints

  Binary variables (9)
    x0 x1 x2 x3 x4 x5 x6 x7 x8



In [34]:
exact_solver = MinimumEigenOptimizer(NumPyMinimumEigensolver())
exact_result = exact_solver.solve(qp)
print(exact_result.prettyprint())

objective function value: -12.0
variable values: x0=0.0, x1=1.0, x2=1.0, x3=0.0, x4=0.0, x5=1.0, x6=0.0, x7=1.0, x8=1.0
status: SUCCESS


In [35]:
grover_optimizer = PBILGroverOptimizer(8, num_iterations=10, quantum_instance=backend, 
                                       pbil_sample_num=1, quantile=0., lr=0.3, )
results = grover_optimizer.solve(qp)
print(results.prettyprint())
print(results.operation_counts)

sampled [1 1 1 1 1 1 1 0 0]
value 21
real_val 21.0
sampled [1 0 1 0 1 0 0 1 1]
value -4
real_val -4.0
param [0.65 0.35 0.65 0.35 0.65 0.35 0.35 0.65 0.65]
threshold -4
sampled [1 0 1 0 1 1 0 1 1]
value -5
real_val -5.0
param [0.755 0.245 0.755 0.245 0.755 0.545 0.245 0.755 0.755]
threshold -5
sampled [1 0 0 1 0 1 0 1 1]
value 0
real_val 0.0
sampled [1 1 0 1 1 0 0 1 1]
value 4
real_val 4.0
sampled [1 0 1 0 1 0 1 1 1]
value 4
real_val 4.0
sampled [1 1 1 1 0 1 0 1 1]
value -7
real_val -7.0
param [0.8285 0.4715 0.8285 0.4715 0.5285 0.6815 0.1715 0.8285 0.8285]
threshold -7
sampled [1 0 1 1 0 1 0 1 0]
value -2
real_val -2.0
sampled [1 0 0 0 0 1 0 1 1]
value 3
real_val 3.0
sampled [1 0 0 0 1 1 0 1 1]
value 4
real_val 4.0
sampled [0 1 1 0 1 1 0 1 1]
value -11
real_val -11.0
param [0.57995 0.63005 0.87995 0.33005 0.66995 0.77705 0.12005 0.87995 0.87995]
threshold -11
sampled [0 1 0 1 1 1 0 1 1]
value 5
real_val 5.0
sampled [0 1 1 1 1 1 0 1 1]
value -6
real_val -6.0
sampled [0 1 1 0 1 1 0 1 1]


In [36]:
def eval_num(operation_dict):
    eval_num = 0
    for x in operation_dict.values():
        eval_num += x['Q(x)']
        if 'Q' in x:
            eval_num += x['Q']
    return eval_num

In [37]:
eval_num(results.operation_counts)

62

In [38]:
grover_optimizer = GroverOptimizer(8, num_iterations=10, quantum_instance=backend,)
results = grover_optimizer.solve(qp)
print(results.prettyprint())
print(results.operation_counts)

objective function value: -12.0
variable values: x0=0.0, x1=1.0, x2=1.0, x3=0.0, x4=0.0, x5=1.0, x6=0.0, x7=1.0, x8=1.0
status: SUCCESS
{0: OrderedDict([('measure', 18), ('h', 9), ('Q(x)', 1)]), 1: OrderedDict([('measure', 18), ('h', 9), ('Q(x)', 1)]), 2: OrderedDict([('measure', 18), ('h', 9), ('Q(x)', 1), ('Q', 1)]), 3: OrderedDict([('measure', 18), ('h', 9), ('Q(x)', 1)]), 4: OrderedDict([('measure', 18), ('h', 9), ('Q(x)', 1)]), 5: OrderedDict([('measure', 18), ('h', 9), ('Q(x)', 1)]), 6: OrderedDict([('measure', 18), ('h', 9), ('Q(x)', 1), ('Q', 1)]), 7: OrderedDict([('measure', 18), ('h', 9), ('Q(x)', 1)]), 8: OrderedDict([('measure', 18), ('h', 9), ('Q', 3), ('Q(x)', 1)]), 9: OrderedDict([('measure', 18), ('h', 9), ('Q(x)', 1)]), 10: OrderedDict([('measure', 18), ('h', 9), ('Q(x)', 1)]), 11: OrderedDict([('measure', 18), ('h', 9), ('Q(x)', 1)]), 12: OrderedDict([('measure', 18), ('h', 9), ('Q(x)', 1), ('Q', 1)]), 13: OrderedDict([('measure', 18), ('h', 9), ('Q(x)', 1), ('Q', 1)]

In [39]:
eval_num(results.operation_counts)

66

objective function value: -57.0
variable values: x0=0.0, x1=1.0, x2=1.0, x3=1.0, x4=1.0, x5=1.0, x6=0.0, x7=1.0, x8=1.0, x9=1.0
status: SUCCESS


In [ ]:
grover_optimizer._pbil_param
grover_optimizer._threshold

In [12]:
objective

docplex.mp.quad.QuadExpr(-0.332x0^2-0.867x0*x1-0.715x0*x2-1.384x0*x3-0.650x0*x4-0.167x0*x5-0.652x0*x6-1.137x0*x7-1.154x0*x8-1.501x0*x9-0.167x1^2-0.950x1*x2-1.070x1*x3-0.809x1*x4-1.145x1*x5-0.446x1*x6-0.326x1*x7-0.954x1*x8-1.285x1*x9-0.646x2^2-0.478x2*x3-0.394x2*x4-1.630x2*x5-0.255x2*x6-0.831x2*x7-1.478x2*x8-1.231x2*x9-0.170x3^2-0.347x3*x4-0.710x3*x5-1.273x3*x6-1.429x3*x7-0.786x3*x8-1.224x3*x9-0.935x4^2-1.334x4*x5-1.073x4*x6-0.728x4*x7-1.189x4*x8-0.887x4*x9-0.666x5^2-0.549x5*x6-0.758x5*x7-0.789x5*x8-0.765x5*x9-0.856x6^2-1.423x6*x7-1.099x6*x8-1.471x6*x9-0.445x7^2-1.026x7*x8-1.879x7*x9-0.080x8^2-1.241x8*x9-0.211x9^2-0.942x0+0.875x1+0.554x2+0.216x3+0.323x4+1.168x5-0.749x6-1.689x7-0.921x8-0.942x9)